In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

gene_set_x_sample = pd.read_csv(PATH["gene_set_x_sample.tsv"], sep="\t", index_col=0)

In [ ]:
make_match_panel_keyword_arguments = dict(
    n_job=SETTING["n_job"],
    n_sampling=SETTING["n_sampling"],
    n_permutation=SETTING["n_permutation"],
    target_type="binary",
    plot_std=SETTING["plot_std"],
)

for target_name, target_values in target_x_sample.iterrows():

    target_values = target_values[target_values != -1]

    match_function = ccal.compute_information_coefficient

    if target_values.value_counts().min() < 3:

        continue

    output_directory_path = os.path.join(
        PATH["find_differentially_expressed_gene_set/"], target_name
    )

    ccal.establish_path(output_directory_path, "directory")

    file_path_prefix = os.path.join(output_directory_path, "all")

    ccal.make_match_panel(
        target_values,
        gene_set_x_sample,
        match_function=match_function,
        n_extreme=SETTING["n_extreme"],
        title="All",
        file_path_prefix=file_path_prefix,
        **make_match_panel_keyword_arguments,
    )

    score_moe_p_value_fdr = pd.read_csv(
        "{}.tsv".format(file_path_prefix), sep="\t", index_col=0
    )

    for gene_set_file_path in SETTING["gene_set_file_paths"]:

        gene_set_name = os.path.split(gene_set_file_path)[1]

        ccal.make_match_panel(
            target_values,
            gene_set_x_sample.reindex(index=ccal.read_gmt(gene_set_file_path).index),
            score_moe_p_value_fdr=score_moe_p_value_fdr,
            n_extreme=SETTING["n_extreme"],
            title=gene_set_name,
            file_path_prefix=os.path.join(output_directory_path, gene_set_name),
            **make_match_panel_keyword_arguments,
        )

    ccal.make_match_panel(
        target_values,
        gene_set_x_sample.reindex(index=SETTING["gene_sets_to_peek"]),
        score_moe_p_value_fdr=score_moe_p_value_fdr,
        n_extreme=None,
        title="Peek",
        **make_match_panel_keyword_arguments,
    )

    ccal.plot_point_and_annotate(
        None,
        score_moe_p_value_fdr["Score"].sort_values(),
        "y",
        annotation=(("Peek", SETTING["gene_sets_to_peek"], 8, "#20d9ba"),),
        title=dict(text=target_name),
        html_file_path=os.path.join(
            output_directory_path,
            "{}.plot_point_and_annotate.html".format(match_function.__name__),
        ),
    )